In [1]:
import sys

sys.path.append('../')

import rings2cosmo
import pandas as pd

### Using 103 objects

In [2]:
# data_full = pd.read_csv('../data/sample103.csv')
# data_full.head()

### Using 80 objects

In [3]:
data_80 = pd.read_csv('../data/sample80.csv')
data_80.head()

,name,zl,zs,sigma_0,dsigma_0,theta_E,theta_ap,sigma_atm,sigma_ap,d_sigma_ap
0,J0151+0049,0.517,1.364,226.0,40.0,0.68,1.0,1.8,219.0,39.0
1,J0747+4448,0.437,0.897,277.0,51.0,0.61,1.0,1.8,281.0,52.0
2,J0830+5116,0.530,1.332,274.0,37.0,1.14,1.0,1.8,268.0,36.0
3,J0944-0147,0.539,1.179,207.0,35.0,0.72,1.0,1.8,204.0,34.0
4,J1215+0047,0.642,1.297,266.0,46.0,1.37,1.0,1.8,262.0,45.0


In [4]:
len(data_80)

80

In [5]:
data = data_80
z_L = data['zl'].values
z_S = data['zs'].values
theta_E = (data['theta_E'].values * rings2cosmo.u.arcsec).to(rings2cosmo.u.rad).value
theta_ap = (data['theta_ap'].values * rings2cosmo.u.arcsec).to(rings2cosmo.u.rad).value
seeing_atm = (data['sigma_atm'].values * rings2cosmo.u.arcsec).to(rings2cosmo.u.rad).value
velDisp = data['sigma_ap'].values
velDispErr = data['d_sigma_ap'].values

In [6]:
rings2cosmo.minimization_loglikelihood(z_S, z_L, velDisp, velDispErr, theta_E, seeing_atm, theta_ap, 
                               seed = 11, alpha_ini = 2.0, beta_ini = 0.18, gamma_ini = 1.0, delta = 2.4)

(1.857292469742536, 0.07619546575730705, 0.743152488980482)

In [7]:
rings2cosmo.minimization_logprobability(z_S, z_L, velDisp, velDispErr, theta_E, seeing_atm, theta_ap, 
                                seed = 11, alpha_ini = 2.0, beta_ini = 0.18, gamma_ini = 1.0, delta = 2.4, \
                                alpha_0_value = 2.0, eps_alpha_0_value = 0.08, beta_0_value = 0.18, eps_beta_0_value = 0.13)

/share/storage1/TNG/Softwares/anaconda3-2020.02/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


array([1.99560323, 0.17999999, 1.02103373])

In [8]:
rings2cosmo.logprobability_sampling(z_S, z_L, velDisp, velDispErr, theta_E, seeing_atm, theta_ap, 
                            seed = 11, alpha_ini = 2.0, beta_ini = 0.18, gamma_ini = 1.0, delta = 2.4, 
                            alpha_0_value = 2.0, eps_alpha_0_value = 0.08, beta_0_value = 0.18, eps_beta_0_value = 0.13, 
                            n_dim = 3, n_walkers = 64, n_burn = 500, n_steps = 1000, progress = True, processes = 10)

Running burn-in ...


100%|████████████████████████████████████████████████████████████| 500/500 [00:10<00:00, 47.90it/s]


Sampling ...


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:20<00:00, 48.19it/s]


Add ploting information...

add hide_print option if progress = False